# Digital Twin Fairways
WIP to use the OpenTNSim engine

In [1]:
import datetime
import json
import logging
import time

import dtv_backend.berthing

# the simpy processes and objects
import dtv_backend.compat

# library to load the fairway information network
import dtv_backend.fis
import dtv_backend.network
import dtv_backend.network.network_utilities
import dtv_backend.postprocessing
import dtv_backend.simple
import dtv_backend.simulate
import geojson
import networkx as nx
import pandas as pd
import shapely
import simpy
from networkx.readwrite import json_graph

# opentnsim import
from opentnsim import core

# reload for debugging purposes
%load_ext autoreload
%autoreload 2

logger = logging.getLogger("notebook")

/Users/baart_f/.virtualenvs/main/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning:

The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.



### Input
You can define your input in a json configuration file. The relevant parts are sites, fleet and climate.

In [2]:
# example input
with open("config.geojson") as f:
    config = geojson.load(f)

### Simulate with ``v3_run``

In [11]:
def v3_dtv_backendulate(config):
    result = dtv_backend.simulate.v3_run(config)
    env = result['env']
    logbook = result['operator'].logbook
    log_df = pd.DataFrame(logbook)
    log_json = dtv_backend.postprocessing.log2json(log_df)
    
    response = {
        "log": log_json,
        "config": config,
        "env": {
            "epoch": env.epoch.timestamp(),
            "epoch_iso": env.epoch.isoformat(),
            "now": env.epoch,
            "now_iso": datetime.datetime.fromtimestamp(env.now).isoformat(),
        },
    }
    return response
    

In [12]:
resp = v3_simulate(config)

In [13]:
resp

{'log': [{'Message': 'Plan',
   'Timestamp': datetime.datetime(2022, 7, 13, 14, 57, 40, 46983),
   'Value': None,
   'geometry': None,
   'ActivityID': 98,
   'ActivityState': None,
   'Meta': {'state': 'START',
    'actor': <dtv_backend.simple.Operator at 0x2b7e30d00>,
    'actor_id': '241496a6-02bc-11ed-95d4-7ebe18db5f99',
    'description': 'Plan (Operator)'}},
  {'Message': 'Task',
   'Timestamp': datetime.datetime(2022, 7, 13, 14, 57, 40, 46983),
   'Value': None,
   'geometry': None,
   'ActivityID': 99,
   'ActivityState': None,
   'Meta': {'state': 'START',
    'actor': <dtv_backend.simple.Operator at 0x2b7e30d00>,
    'actor_id': '241496a6-02bc-11ed-95d4-7ebe18db5f99',
    'description': 'Sending task (Operator)',
    'max_load': 2286,
    'source': <dtv_backend.compat.Port at 0x2b7e30910>,
    'destination': <dtv_backend.compat.Port at 0x2b7e30580>}},
  {'Message': 'Task',
   'Timestamp': datetime.datetime(2022, 7, 13, 15, 57, 40, 46983),
   'Value': None,
   'geometry': None